## Mathematik für Biologiestudierende II

Sommersemester 2024

25.06.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf
import seaborn as sns
sns.set_theme()
import warnings
warnings.filterwarnings('ignore', message='The figure layout has changed')

# Lineare Modelle

#### Themen heute

* mehrere erklärende Variablen
* Transformationen
* Normalverteilungsannahmen
* kategorielle erklärende Variablen

# Mehrere erklärende Variable

In [ ]:
df = pd.read_csv('larven.csv')
df.head()

* Anzahl_Larven:  Anzahl der Larven eines Kleinstlebewesens pro Liter
* A, B, C, D, E:  Konzentrationen von fünf potentiellen Schadstoffen in ppb (Teile pro Milliarde)

* das Experiment ist *beobachtend*
* keine Möglichkeit der unabhängigen Veränderung einzelner Parameter

In [ ]:
df.describe().round(0)

In [ ]:
formel = 'Anzahl_Larven ~ A + B + C + D + E'
modell = smf.ols(formel, df)

In [ ]:
res = modell.fit()

In [ ]:
res.summary()

* Der Einfluss von B und E ist nicht signifikant
* Wir entfernen sie einzeln aus dem Modell
* Am wenigsten signifikant ist der Einfluss von E

In [ ]:
formel3 = 'Anzahl_Larven ~ A + B + C + D'
modell3 = smf.ols(formel3, df)
res3 = modell3.fit()

In [ ]:
res3.summary()

* der Einfluss von B ist immer noch nicht signifikant

In [ ]:
formel4 = 'Anzahl_Larven ~ A + C + D'
modell4 = smf.ols(formel4, df)
res4 = modell4.fit()

In [ ]:
res4.summary()

* die drei verbleibenden Stoffe haben signifikanten Einfluss auf die Larvenpopulation
* A und D verringen die Anzahl: Schadstoffe
* C erhöht sie:  Nährstoff

# Transformationen

* Das Konfidenzintervall für den Koeffizienten des Stoffs D wird angegeben als [-0., -0.]
* Lösung:  wir geben die Konzentration von D statt in ppb in ppm (parts per million) an
* Das multipliziert den Koeffizienten mit 1000

In [ ]:
df['D_in_ppm'] = df.D / 1000

In [ ]:
formel5 = 'Anzahl_Larven ~ A + C + D_in_ppm'
modell5 = smf.ols(formel5, df)
res5 = modell5.fit()

In [ ]:
res5.summary()

## Transformation

* wir haben eine Spalte der Tabelle transformiert von ppb auf ppm
* dadurch wurde die Statistik nicht verändert; das Ergebnis wurde aber anschaulicher

* einzelne Zeilen können nicht transformiert werden, alle Zeilen müssen in demselbe System gemessen werden

* Transformationen werden auch verwendet, wenn Daten die Anwendungsvoraussetzunge nicht erfüllen

## Beispiel: Galapagos-Inseln

* Daten aus Faraway:  Linear Models with Python
* ursprüngliche Datenquelle
  * Johnson, M., and Raven, P.:  *Species Number and endemism: the Gálapagos Archipelago revisited.*  Science 179 (1973), 893-895

In [ ]:
df = pd.read_csv('galapagos.csv')
df.head()

* Species:  Anzahl verschiedener Wirbeltierarten
* Area:  Größe der Insel
* Elevation:  Höchste Erhebung auf der Insel
* Nearest:  Abstand zur nächsten Insel
* Scruz:  Abstand zu Santa Cruz
* Adjacent:  Größe der nächstgelegenen Insel

In [ ]:
df.describe()

In [ ]:
formel = 'Species ~ Area + Elevation + Nearest + Scruz + Adjacent'
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

Nur zwei der erklärenden Variablen haben signifikanten Einfluss

* Die Höhe der Insel
* Die Fläche der Nachbarinsel mit negativer Korrelation ❓❓❓

# Normalverteilungsannahmen

* `smf.ols` hat Anwendungsvoraussetzungen
* eine davon ist, dass alle Variablen normalverteilt sind

* wir prüfen das mit qq-Plots wie in Lektion 14

In [ ]:
import statsmodels.api as sm
pp_s = sm.ProbPlot(df.Species)
pp_s.qqplot();

* eine Transformation ist notwendig

* Der Logarithmus ist eine mögliche Wahl
* So machen das auch Johnson und Raven

In [ ]:
pp_lw = sm.ProbPlot(np.log(df.Species))
pp_lw.qqplot();

Ich zeige alle QQ-Plots in einem Bild mit einem Verfahren, welches nicht zum Stoff gehört

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()
for j in range(6):
    spalte = df.columns[j+1]
    ax = fig.add_subplot(231+j)
    pp = sm.ProbPlot(df[spalte])
    pp.qqplot(ax=ax, xlabel=spalte, ylabel='')
fig.subplots_adjust(wspace=0.5, hspace=0.4);

* Also verletzen alle Variablen die Normalverteilungsannahme mehr oder weniger deutlich
* Wir transformieren sie daher ebenfalls
* Problem:  Scruz hat Abstand 0 von sich selbst
* $\log$ ist nur für positive Zahlen erklärt

* Wir entfernen diese Spalte aus dem Modell

In [ ]:
fig = plt.figure()
for j in range(6):
    if j != 4:
        spalte = df.columns[j+1]
        ax = fig.add_subplot(231+j)
        pp = sm.ProbPlot(np.log(df[spalte]))
        pp.qqplot(ax=ax, xlabel=spalte, ylabel='')
fig.subplots_adjust(wspace=0.5, hspace=0.4);

In [ ]:
transformierte_formel = '''np.log(Species) ~ np.log(Area) + np.log(Elevation) 
                                         + np.log(Nearest) + np.log(Adjacent)'''
modell2 = smf.ols(transformierte_formel, df)

`'''` Zeichenkette (engl "strings") über mehrere Zeilen

In [ ]:
res2 = modell2.fit()
res2.summary()

Nur noch die Fläche hat einen signifikanten Einfluss

# Kategorielle Daten

In [ ]:
df = pd.read_csv('kinder.csv')
df.head()

In [ ]:
sns.lmplot(df, x='father', y='childHeight', hue='gender');

In [ ]:
formel = 'childHeight ~ father + mother + gender'
modell = smf.ols(formel, df)

In [ ]:
res = modell.fit()

In [ ]:
res.summary()

Hier wird eine Fallunterscheidung kodiert

$$
    \text{childHeight} = 16.5212 + 0.3928 \cdot \text{father} + 0.13176 \cdot \text{mother} + 
    \begin{cases}
        0.0 & \text{wenn Mädchen,} \\
        5.215 & \text{wenn Junge.}
    \end{cases}
$$

Die Terminologie kommt offenbar aus der Pharmazie:

* `female` ist hier der Standard (engl. "default")
* alles, was vom Standard abweicht, ist eine Behandlung (engl. "treatment")
* was default und was treatment ist, entscheidet das Programm